### This is designed to merge the single papers together to assemble one bigger data-csv that we can at some point use for model training

TODO: 
Investigate: 
https://www.nature.com/articles/s41598-023-41443-4?fromPaywallRec=false 
https://www.nature.com/articles/s41593-023-01514-1.pdf 


# General Notes: 
- To execute the notebook, you frist need to produce the .csv from the underlying papers
- I try to convert and select for ENSG gene rows as we got  datasets from Flo with 60k entries which most probably will be me mayjority


In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from scipy.io import mmread
from utils.plotting import manhattanplot, violinplot_overall, scatter_plot
from utils.helper import keep_latest_ensamble_version, get_negative_values, METADATA_COLS, reorder_columns_by_metadata_and_gene_counts, convert_gene_names_to_ensembl, clean_age
from utils.filepath import *
%autoreload 2



In [ ]:
### Wimmers 2023 (NOT INCLUDED)
#NOTE This is 1 of 2 Datasets from this paper. 
#NOTE Also this datasets should be clarified with Flo first. It´s PBMC but not clear from whom ? (maybe GSE239799 ??? but also different)
wimmers_2_df = pd.read_csv(f'{PATH_WIMMERS2023_COMBI}')
print(wimmers_2_df.shape)
PLATFORM_GEO_ID_wimmers_23 = "GPL24676"
wimmers_2_df['PLATFORM_GEO_ID']=PLATFORM_GEO_ID_wimmers_23

PLATFORM_DESCRIPTION_wimmers_23 = "Illumina NovaSeq 6000 (Homo sapiens)"
wimmers_2_df['PLATFORM_DESCRIPTION']=PLATFORM_DESCRIPTION_wimmers_23

METHOD_wimmers_23 = "RNAseq"
wimmers_2_df['METHOD']=METHOD_wimmers_23

#TODO in the data itself PBMC is mentioned but https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE239787 it is whole blood 
TYPE_wimmers_23 = "blood"
wimmers_2_df['TYPE']=TYPE_wimmers_23

desease_wimmers_23 = "SARS-CoV-2 / healthy"
wimmers_2_df['desease']=desease_wimmers_23

wimmers_2_df.head()

In [ ]:
### Wimmers 2023 GEO239787
#NOTE This is 2 of 2 Datasets from this paper. The GEO Is more useable as it contains what we need
wimmers_GEO239787_df = pd.read_csv(PATH_WIMMER2023_GEO, index_col=0)


wimmers_GEO239787_df = wimmers_GEO239787_df.rename(columns={
    'ID':'subject_accession', 
  'geo_accession':'data_accession',
    'characteristics_ch1.4':'age',
    'characteristics_ch1.3':'gender',
    'Status':'desease', 
  'contact_city':'origin',
   'library_strategy':'METHOD', 
   'source_name_ch1':'TYPE',
   'instrument_model':'PLATFORM_DESCRIPTION',
   'platform_id':'PLATFORM_GEO_ID'
})

wimmers_GEO239787_df['study_accession']='GSE239787'
wimmers_GEO239787_df['origin']='Contact City: Palo Alto'
wimmers_GEO239787_df['age']=wimmers_GEO239787_df['age'].str.replace('age, months: ', '').astype(float)
wimmers_GEO239787_df['age']=wimmers_GEO239787_df['age']/12

wimmers_GEO239787_df_reduced = reorder_columns_by_metadata_and_gene_counts(wimmers_GEO239787_df)

print(wimmers_GEO239787_df_reduced.shape)
wimmers_GEO239787_df_reduced.tail()

In [ ]:
# 2020 Arunachalam, Wimmers
#NOTE migh had some issues in the prep df file, but the GEO actually did a good job 
#NOTE I can probably add 4 more subjects but want to clarify with flo first
aranachalam_2020_GSE152418_df = pd.read_csv(PATH_ARUNACHALAM2020_FINALDESTINATION, index_col=0)
print(aranachalam_2020_GSE152418_df.shape)
aranachalam_2020_GSE152418_df = aranachalam_2020_GSE152418_df.rename(columns={
  'title':'subject_accession', 
  'geo_accession':'data_accession',
  'Age':'age',
  'characteristics_ch1.2':'gender',
  'characteristics_ch1.3':'desease', 
  'contact_city':'origin',
  'library_strategy':'METHOD', 
  'source_name_ch1':'TYPE',
  'instrument_model':'PLATFORM_DESCRIPTION',
  'platform_id':'PLATFORM_GEO_ID',
})
aranachalam_2020_GSE152418_df['study_accession'] = 'GSE152418'
aranachalam_2020_GSE152418_df = reorder_columns_by_metadata_and_gene_counts(aranachalam_2020_GSE152418_df)
print(aranachalam_2020_GSE152418_df.shape)
aranachalam_2020_GSE152418_df.head()

In [ ]:
# 2021 Thair COVID 
# NOTE 

thair_2021_GSE152641_df = pd.read_csv(f"{BASE_PATH}/2021_thair/thair_combi_covid.csv", low_memory=False, index_col=0)
thair_2021_GSE152641_df = thair_2021_GSE152641_df.loc[1:86, :] # there are 3 meta rows in the csv... 
print(thair_2021_GSE152641_df.shape)
thair_2021_GSE152641_df = thair_2021_GSE152641_df.rename(columns={
  'title':'subject_accession', 
    'geo_accession':'data_accession',
    'characteristics_ch1.3':'age',
    'characteristics_ch1.4':'gender',
    'characteristics_ch1':'desease',
  'contact_city':'origin',
  'library_strategy':'METHOD',
  'characteristics_ch1.2':'TYPE',
  'instrument_model':'PLATFORM_DESCRIPTION',
  'platform_id':'PLATFORM_GEO_ID',
})
thair_2021_GSE152641_df['study_accession'] ='GSE152641'
thair_2021_GSE152641_ordered_df = reorder_columns_by_metadata_and_gene_counts(thair_2021_GSE152641_df)
print(thair_2021_GSE152641_ordered_df.shape)

columns_to_cast = thair_2021_GSE152641_ordered_df.columns[11:]
thair_2021_GSE152641_ordered_df[columns_to_cast] = thair_2021_GSE152641_ordered_df[columns_to_cast].astype(float)


thair_2021_GSE152641_ordered_df.head()


In [ ]:
# 2021 Aranachalam VACCINE
#NOTE Had to convert Gene names to ENSEMBL which sometimes results in a not found version -> Flo how to handle 
#NOTE Vaccine study, i just took the Baseline (blood before the vaccine was given)

import copy

aranachalam_2021_GSE169159_df = pd.read_csv(f'{BASE_PATH}/2021_arunachalam/2021_aranachalam_vaccine_combi_df_baseline.csv')
print(aranachalam_2021_GSE169159_df.shape)
aranachalam_2021_GSE169159_df = aranachalam_2021_GSE169159_df.rename(columns={
  'title':'subject_accession', 
  'geo_accession':'data_accession',
  'age':'age',
    'sex':'gender',
  'contact_city':'origin',
  'library_strategy':'METHOD',
  'source_name_ch1':'TYPE',
  'instrument_model':'PLATFORM_DESCRIPTION',
  'platform_id':'PLATFORM_GEO_ID',
})
aranachalam_2021_GSE169159_df['desease']='Healthy'
aranachalam_2021_GSE169159_df['study_accession']='GSE169159'

## This is weird as the overwrite of a subset of columns of a dataframe seems not to work. I tried stuff in different patterns here
print(f"First gene: {aranachalam_2021_GSE169159_df.columns.get_loc('FGR')}")
gene_names_to_convert = aranachalam_2021_GSE169159_df.iloc[:, 38:-2].copy().columns
aranachalam_2021_GSE169159_ordered_df = reorder_columns_by_metadata_and_gene_counts(aranachalam_2021_GSE169159_df, 
                                                                                    metadata_cols=METADATA_COLS,
                                                                                    gene_columns=gene_names_to_convert)
print(f"First gene: {aranachalam_2021_GSE169159_ordered_df.columns.get_loc('FGR')}")
converted_gene_names = convert_gene_names_to_ensembl(aranachalam_2021_GSE169159_ordered_df.iloc[:, 11:].copy().columns)

# Now i want to overwrite every column of the dataset (also i dont need to use loc or iloc)
all_dataset_new_columns = METADATA_COLS + converted_gene_names
aranachalam_2021_GSE169159_ordered_df.columns = all_dataset_new_columns

print(aranachalam_2021_GSE169159_ordered_df.columns[60])
print(converted_gene_names[49]) # as i need to add the meta cols ... 

print(aranachalam_2021_GSE169159_ordered_df.shape)
aranachalam_2021_GSE169159_ordered_df.tail()

In [ ]:
# 2023 feargal
# '/work/transcriptomics_norm_model/2023_feargal/2023_feargal_combi_df.csv'
feargal_2023_GSE199750_df_init = pd.read_csv('/work/transcriptomics_norm_model/2023_feargal/2023_feargal_combi_df.csv')
print(feargal_2023_GSE199750_df_init.shape)
# HERE IS A BUG, the data got age and gender cols confused in some rows... 
feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.2'] = feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.3']
feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.3'] = feargal_2023_GSE199750_df_init.loc[:31, 'characteristics_ch1.4']

In [ ]:

feargal_2023_GSE199750_df_init['characteristics_ch1.3'].unique()


In [ ]:

feargal_2023_GSE199750_df = feargal_2023_GSE199750_df_init.rename(columns={
        'characteristics_ch1':'subject_accession', 
           'geo_accession':'data_accession',
           'characteristics_ch1.2':'age',
          'characteristics_ch1.3':'gender',
          'contact_city':'origin',
           'library_strategy':'METHOD',
           'source_name_ch1':'TYPE',
          'instrument_model':'PLATFORM_DESCRIPTION',
          'platform_id':'PLATFORM_GEO_ID',
})
feargal_2023_GSE199750_df['desease']='Healthy'
feargal_2023_GSE199750_df['study_accession']='GSE199750'
feargal_2023_GSE199750_reordered_df = reorder_columns_by_metadata_and_gene_counts(feargal_2023_GSE199750_df)
print(feargal_2023_GSE199750_reordered_df.shape)
feargal_2023_GSE199750_reordered_df.head()

In [ ]:
#SOARES-SCHANINSKI
#NOTE CHIKV virus patients and ~20 healthy controls

soares_df = pd.read_csv('/work/transcriptomics_norm_model/2019_soares/2019_soares_combi_df.csv')
print(soares_df.shape)
soares_df = soares_df.rename(columns={
                'BioSample':'subject_accession', 
                'Run':'data_accession',
                'Age':'age',
                'sex':'gender',
                'Center Name':'origin',
                'Assay Type':'METHOD',
                'tissue':'TYPE',
                'Instrument':'PLATFORM_DESCRIPTION',
                'disease': 'desease'
})
soares_df['study_accession']='BioProject'
soares_df['PLATFORM_GEO_ID']='-'
soares_df = reorder_columns_by_metadata_and_gene_counts(soares_df)
print(soares_df.shape)
soares_df.head()

In [ ]:
#O'Connell
#NOTE Different deseased patients

oconnell_df = pd.read_csv(f'{OCONNELL2023_COMBI}')
print(oconnell_df.shape)
oconnell_df = oconnell_df.rename(columns={
                 'index':'subject_accession', 
                 'BioSample':'study_accession',
                 'Run':'data_accession',
                 'AGE':'age',
                 'sex':'gender',
                 'Center Name':'origin',
                 'Assay Type':'METHOD',
                 'tissue':'TYPE',
                 'Instrument':'PLATFORM_DESCRIPTION',
                 'disease': 'desease',
})

oconnell_df['PLATFORM_GEO_ID']='GPL24676'
oconnell_df = reorder_columns_by_metadata_and_gene_counts(oconnell_df)
print(oconnell_df.shape)
oconnell_df.head()


In [ ]:
# Dr. Susanne Edelmann
# NOTE: from Psychiatry , found its GEO Entries: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE228702 
edelmann_df = pd.read_csv(EDLMANN2023_COMBI)
print(edelmann_df.shape)
edelmann_df = edelmann_df.rename(columns={
                   'seq_name':'subject_accession', 
                    'AGE':'age',
                    'SEX':'gender',
                    'SAD': 'desease',
})
edelmann_df['PLATFORM_DESCRIPTION'] = 'Illumina NovaSeq 6000'
edelmann_df['data_accession']  = 'SAMN34035152'
edelmann_df['study_accession'] = 'GSE228702'
edelmann_df['TYPE'] = 'whole blood'
edelmann_df['METHOD'] = 'RNAseq'
edelmann_df['origin'] = 'Department of Psychiatry and Psychotherapy, University Hospital of Tuebingen'
edelmann_df['PLATFORM_GEO_ID']='GPL24676'
edelmann_df = reorder_columns_by_metadata_and_gene_counts(edelmann_df)

edelmann_df.head()


In [ ]:
## NOTE PPMI, i might move this to a different notebook tho... 
from utils.filepath import *

ppmi_ir3_baseline_df = pd.read_csv(PPMI_IR3_COMBI_BL)
print(ppmi_ir3_baseline_df.shape)
ppmi_ir3_baseline_df.head()

ppmi_ir3_baseline_df = ppmi_ir3_baseline_df.rename(columns={
                      'PATNO':'subject_accession', 
                      'Sample':'data_accession',
                      'ENROLL_AGE':'age',
                      'GENDER':'gender',
                      'DIAGNOSIS': 'desease',
})
ppmi_ir3_baseline_df['PLATFORM_DESCRIPTION'] = 'Illumina NovaSeq 6000'
ppmi_ir3_baseline_df['PLATFORM_GEO_ID']='GPL24676'
ppmi_ir3_baseline_df['study_accession'] = 'IR3_p1+p2'
ppmi_ir3_baseline_df['METHOD'] = 'RNAseq'
ppmi_ir3_baseline_df['TYPE'] = 'whole blood'
ppmi_ir3_baseline_df['origin'] = 'PPMI'
ppmi_ir3_baseline_df = reorder_columns_by_metadata_and_gene_counts(ppmi_ir3_baseline_df)

In [ ]:
#NOTE: This cell is there to take the latest version of the gene count, without this cell we would have douplicate column names, what makes concat impossible

## As PPMI uses version identifiesers (ENSG00000243485.5 the .5 part...) we need to trim these, when there are duplicates afterwards, i take the latest version
ppmi_ir3_baseline_w_duplicates_df = copy.copy(ppmi_ir3_baseline_df)
ppmi_ir3_baseline_w_duplicates_df.columns = [col.split('.')[0] for col in ppmi_ir3_baseline_w_duplicates_df.columns]
duplicates = ppmi_ir3_baseline_w_duplicates_df.columns[ppmi_ir3_baseline_w_duplicates_df.columns.duplicated()]
print(duplicates)    # expected 0 duplicates as version nr are still there

ppmi_ir3_baseline_wo_duplicates_df = keep_latest_ensamble_version(ppmi_ir3_baseline_df, duplicates=duplicates)
ppmi_ir3_baseline_wo_duplicates_df.columns = [col.split('.')[0] for col in ppmi_ir3_baseline_wo_duplicates_df.columns]
print(ppmi_ir3_baseline_wo_duplicates_df.columns[ppmi_ir3_baseline_wo_duplicates_df.columns.duplicated()])
ppmi_ir3_baseline_wo_duplicates_df.head()

In [ ]:
ppmi_ir3_baseline_wo_duplicates_df.groupby('desease').count()


In [ ]:
concatinated_df = pd.concat([aranachalam_2020_GSE152418_df, 
                             wimmers_GEO239787_df_reduced, 
                             thair_2021_GSE152641_ordered_df, 
                             aranachalam_2021_GSE169159_ordered_df, 
                             feargal_2023_GSE199750_reordered_df,
                             oconnell_df,
                             edelmann_df,
                             soares_df,
                             ppmi_ir3_baseline_wo_duplicates_df
                             ])
# missing is wimmers_2 as i need clarification there
#                             ppmi_ir3_baseline_wo_duplicates_df,

concatinated_df = concatinated_df.reset_index(drop=True)
print(concatinated_df.shape)
concatinated_df.head()

In [ ]:
#concatinated_df.to_csv(f'{BASE_PATH}/combi_df_concat_10.10.24.csv')
#concatinated_df = pd.read_csv(f'{BASE_PATH}/combi_df_concat_10.10.24.csv')

In [ ]:
#NOTE proof if any dta col is not of float type
#Problem1 cant convert 'IMX_sample00001'
columns_to_cast = concatinated_df.columns[11:]
for column in columns_to_cast:
    try:
        concatinated_df[column] = pd.to_numeric(concatinated_df[column], errors='raise')  # Will throw an error if there's an issue
    except ValueError:
        print(f"Column '{column}' contains non-numeric values.")
### Solution: I concat thairs non-ordered df....

#concatinated_df.drop_na(axis=1, how='all')


### Unify the different valuee 

In [ ]:
mapping = {
    'gender: F': 'Female',
    'gender: M': 'Male',
    'Sex: Male': 'Male',
    'Sex: Female': 'Female',
    'female': 'Female',
    'male': 'Male',
}
concatinated_df['gender'] = concatinated_df['gender'].replace(mapping)
concatinated_df['gender'].unique()


In [ ]:

# Apply the function and convert column to float
concatinated_df['age'] = concatinated_df['age'].apply(clean_age).astype(float)

In [ ]:
#NOTE: Oconnell study has several deseases in the field separated by ','
mapping = {
    'disease state: COVID-19': 'Sars-CoV-2',
    'disease: COVID19': 'Sars-CoV-2',
    'acute': 'Sars-CoV-2',
    'acute-omicron': 'Sars-CoV-2',
    'disease state: Healthy': 'Healthy',
    'healthy': 'Healthy',
    'disease: Healthy control': 'Healthy',
    'Healthy': 'Healthy',
    'Control': 'Healthy',
}
concatinated_df['desease'] = concatinated_df['desease'].replace(mapping)
concatinated_df['desease'].unique()

In [ ]:
#Filter for duplicated subjects. Here just the same subj ID is most likely not sufficient (check those "draws and so on ")
print(len(concatinated_df['subject_accession'].unique()))
concatinated_df_per_subject = concatinated_df.drop_duplicates(subset=['subject_accession'])
concatinated_df_per_subject = concatinated_df_per_subject[~concatinated_df_per_subject['subject_accession'].str.endswith('Draw-2', na=False)]
print(concatinated_df_per_subject.shape)


In [ ]:
#concatinated_df_per_subject.to_csv(f'{BASE_PATH}/concatinated_df_per_subject_11.10.24.csv')
